<a href="https://colab.research.google.com/github/okechukwuchude/Automating-Medical-Coding/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import itertools
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
#ICD codes and meaningg
D_ICD_DIAG = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/D_ICD_DIAGNOSES.csv.gz',compression='gzip')
D_ICD_PROC = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/D_ICD_PROCEDURES.csv.gz',compression='gzip')

In [3]:
#original ICD files
diagnoses_icd = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/DIAGNOSES_ICD.csv.gz',compression='gzip')
procedures_icd = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/PROCEDURES_ICD.csv.gz',compression='gzip')

In [4]:
diagnoses_icd.head(5)

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,4,3,145834,1.0,0389
4,5,3,145834,2.0,78559


In [6]:
#Read notes source file
NOTES = pd.read_csv('/content/drive/MyDrive/mimic-iii-clinical-database-carevue-subset-1.4/NOTEEVENTS.csv.gz', compression='gzip')
KEEP = NOTES[['hadm_id','category','text']]

In [7]:
len(diagnoses_icd['icd9_code'].unique())


5054

In [9]:
KEEP

,hadm_id,category,text
0,NaN,Radiology,[**2119-1-4**] 12:59 PM\n ABDOMEN U.S. (COMPLE...
1,NaN,Radiology,[**2119-1-9**] 1:05 PM\n MR LIVER WITH CONTRAS...
2,NaN,Radiology,[**2119-1-16**] 9:24 PM\n CHEST (PORTABLE AP) ...
3,NaN,Radiology,[**2119-1-18**] 1:24 PM\n CT ABD W&W/O C; CT P...
4,NaN,Radiology,[**2119-1-18**] 3:45 PM\n PARACENTESIS DIAG. O...
...,...,...,...
880102,104049.0,Nursing/other,1. FEN\nTF= min140cc/k/d of BM/E24. min47cc q4...
880103,104049.0,Nursing/other,I have examined pt. & agree w/ [**First Name8 ...
880104,104049.0,Nursing/other,NPN nights\n\n\nFluids/Nutrition: Weight 2025...
880105,104049.0,Nursing/other,Attending Note\nDay of life 12 PMA 35 [**2-7**...
